# Experiments with postgis

Here the goal is to check if we can use postgis as our geometrical/geographical indexing engine

In [ ]:
import psycopg2 as psy
import pandas as pd

In [ ]:
from osgeo import ogr
from osgeo import osr

source = osr.SpatialReference()
source.ImportFromEPSG(4326)

target = osr.SpatialReference()
target.ImportFromEPSG(3003)

transform = osr.CoordinateTransformation(source, target)

def map_to_monte_mario(wkt):
    geom = ogr.CreateGeometryFromWkt(wkt)
    geom.Transform(transform)
    return geom.ExportToWkt()

In [ ]:
geom = ogr.CreateGeometryFromWkt('POINT(9.1092760 39.2284606)')

In [ ]:
print(map_to_monte_mario('POINT(9.1092760 39.2284606)'))
POLY = "POLYGON ((9.147340 39.239940, 9.147380 39.239940, 9.147380 39.239960, 9.147340 39.239980, 9.147340 39.239940))"
print(map_to_monte_mario(POLY))

In [ ]:
# create testdb
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
con = psy.connect("dbname=postgres host=timescaledb user=postgres password=foobar")

with con:
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    with con.cursor() as cur:
        cur.execute("DROP DATABASE IF EXISTS testdbgis;")
        cur.execute("CREATE DATABASE testdbgis;")


In [ ]:
con = psy.connect("dbname=testdbgis host=timescaledb user=postgres password=foobar")

with con:
    with con.cursor() as cur:   
        cur.execute("CREATE EXTENSION IF NOT EXISTS postgis;")
        cur.execute("SELECT postgis_version();")
        print(cur.fetchall())
        cur.execute("SELECT srid, auth_name, proj4text FROM spatial_ref_sys LIMIT 3;")
        for x in cur.fetchall():
            print(x)
        cur.execute("SELECT srid, auth_name, proj4text FROM spatial_ref_sys WHERE srid = 3003")
        print(cur.fetchall())
con.close()

## Saving point-probes location

Simplest scenario, we are just saving sensors positions.

In [ ]:
sensors = pd.read_csv('./data/sensor_data_denorm.csv', index_col='id')

In [ ]:
sensors.head(n=2)

In [ ]:
con = psy.connect("dbname=testdbgis host=timescaledb user=postgres password=foobar")

with con:
    with con.cursor() as cur:
        cur.execute("""
            DROP TABLE IF EXISTS sensors;
            CREATE TABLE sensors (
              id int4 primary key,
              name VARCHAR(20),
              note VARCHAR(100),
              station_id int4,
              sensor_type int4,
              geom geometry(POINT,3003) );
        """)
con.close()

In [ ]:
SQL = """INSERT INTO sensors (id, name, note, station_id, sensor_type, geom)
         VALUES (%s, %s, %s, %s, %s, ST_GeomFromText(%s, 3003));"""

con = psy.connect("dbname=testdbgis host=timescaledb user=postgres password=foobar")
with con:
    with con.cursor() as cur:
        for i, v in zip(sensors.index, sensors.values):
            (name, note, st_id, stype, lat, lon) = v[:-1]
            cur.execute(SQL, (i, name, 
                              note, st_id, stype, 
                              map_to_monte_mario("POINT (%s %s)" % (lon, lat))))
con.close()

In [ ]:
con = psy.connect("dbname=testdbgis host=timescaledb user=postgres password=foobar")
SQL = """
SELECT * 
FROM sensors 
WHERE ST_Contains(ST_GEOMFROMTEXT(%s, 3003), sensors.geom);
"""
RECT = "POLYGON((9.147340 39.239940, 9.147380 39.239940, 9.147380 39.239960, 9.147340 39.239980, 9.147340 39.239940))"

with con:
    with con.cursor() as cur:
        cur.execute(SQL, (map_to_monte_mario(RECT),))
        for x in cur.fetchall():
            print(x)

## Saving area-probes location

In [ ]:
con = psy.connect("dbname=testdbgis host=timescaledb user=postgres password=foobar")

with con:
    with con.cursor() as cur:
        cur.execute("""
            DROP TABLE IF EXISTS areaprobe;
            CREATE TABLE areaprobe (
              id int4 primary key,
              name VARCHAR(20),
              sensor_type int4,
              geom geometry(POLYGON,3003) );
        """)
con.close()

In [ ]:
SQL = """INSERT INTO areaprobe (id, name, geom)
         VALUES (%s, %s, ST_GeomFromText(%s, 3003));"""
RECT = "POLYGON((8.7519483  39.5057942, 8.7519483  38.9511270, 9.4666037  39.5057942,9.4666037  38.9511270, 8.7519483  39.5057942))"


con = psy.connect("dbname=testdbgis host=timescaledb user=postgres password=foobar")
with con:
    with con.cursor() as cur:
        cur.execute(SQL, (12, 'Radar@UNICA', map_to_monte_mario(RECT)))
con.close()

In [ ]:
con = psy.connect("dbname=testdbgis host=timescaledb user=postgres password=foobar")
with con:
    with con.cursor() as cur:
        cur.execute("SELECT * from areaprobe")
        for x in cur.fetchall():
            print(x)
con.close()

## Generic geo objects

In [ ]:
con = psy.connect("dbname=testdbgis host=timescaledb user=postgres password=foobar")

with con:
    with con.cursor() as cur:
        cur.execute("""
            DROP TABLE IF EXISTS geothings;
            CREATE TABLE geothings (
              id int4 primary key,
              name VARCHAR(20),
              type int4,
              geom geometry);
        """)
con.close()

In [ ]:
SQL = """INSERT INTO geothings (id, name, type, geom)
         VALUES (%s, %s, %s, ST_GeomFromText(%s, 3003));"""

con = psy.connect("dbname=testdbgis host=timescaledb user=postgres password=foobar")
with con:
    with con.cursor() as cur:
        for i, v in zip(sensors.index, sensors.values):
            (name, note, st_id, stype, lat, lon) = v[:-1]
            cur.execute(SQL, (i, name, stype, 
                              map_to_monte_mario("POINT (%s %s)" % (lon, lat))))
            
con.close()

In [ ]:
SQL = """INSERT INTO geothings (id, name, type, geom)
         VALUES (%s, %s, %s, ST_GeomFromText(%s, 3003));"""
RECT = "POLYGON((8.7519483  39.5057942, 8.7519483  38.9511270, 9.4666037  39.5057942,9.4666037  38.9511270, 8.7519483  39.5057942))"

con = psy.connect("dbname=testdbgis host=timescaledb user=postgres password=foobar")
with con:
    with con.cursor() as cur:
        cur.execute(SQL, (2000, 'Radar@UNICA', 4, map_to_monte_mario(RECT)))
con.close()

In [ ]:
con = psy.connect("dbname=testdbgis host=timescaledb user=postgres password=foobar")
with con:
    with con.cursor() as cur:
        cur.execute("""
        SELECT name, ST_AsText(ST_Transform(geom,4326)) from geothings
        WHERE ST_DWithin(geom, ST_GeomFromText(%s, 3003), 1000);
        """, (map_to_monte_mario('POINT(9.1092760 39.2284606)'), ))
        for x in cur.fetchall():
            print(x)
con.close()